# Lab 9: SLM-Powered Interpretation of Apriori-Based Association Rules 

```bash
# install required packages
!pip install mlxtend transformers
```

## Step 1: Import and Prepare Transaction Data 

In [9]:
# import libraries
import pandas as pd 
from mlxtend.preprocessing import TransactionEncoder 

In [10]:
dataset = [ 
['Milk', 'Bread', 'Butter'], 
['Milk', 'Bread'], 
['Milk', 'Coke'], 
['Bread', 'Coke'], 
['Bread', 'Butter'] 
] 

te = TransactionEncoder() 
te_data = te.fit(dataset).transform(dataset) 
df = pd.DataFrame(te_data, columns=te.columns_) 

## Step 2: Apply Apriori Algorithm with Lower Thresholds

In [11]:
from mlxtend.frequent_patterns import apriori, association_rules 

frequent_itemsets = apriori(df, min_support=0.2, use_colnames=True) 
rules = association_rules(frequent_itemsets, metric="confidence", 
min_threshold=0.3) 

print(" Frequent Itemsets:\n", frequent_itemsets) 
print("\n Association Rules:\n", rules)

 Frequent Itemsets:
    support               itemsets
0      0.8                (Bread)
1      0.4               (Butter)
2      0.4                 (Coke)
3      0.6                 (Milk)
4      0.4        (Bread, Butter)
5      0.2          (Coke, Bread)
6      0.4          (Bread, Milk)
7      0.2         (Milk, Butter)
8      0.2           (Coke, Milk)
9      0.2  (Milk, Bread, Butter)

 Association Rules:
         antecedents      consequents  antecedent support  consequent support  \
0           (Bread)         (Butter)                 0.8                 0.4   
1          (Butter)          (Bread)                 0.4                 0.8   
2            (Coke)          (Bread)                 0.4                 0.8   
3           (Bread)           (Milk)                 0.8                 0.6   
4            (Milk)          (Bread)                 0.6                 0.8   
5            (Milk)         (Butter)                 0.6                 0.4   
6          (Butter)    

## Step 3: Use SLM (Small Language Model) to Explain Rules 

Using Gemma-3 by Google

In [12]:
# configure api
from dotenv import load_dotenv
import os

load_dotenv()
gemini_api_key = os.getenv("GEMINI_API_KEY")

In [13]:
from google import genai

client = genai.Client(api_key=gemini_api_key)

model = "gemma-3-27b-it"

## Step 4: Generate Explanations

In [14]:
from IPython.display import display, Markdown

if not rules.empty: 
    for index, row in rules.iterrows(): 
        antecedent = ', '.join(list(row['antecedents'])) 
        consequent = ', '.join(list(row['consequents'])) 
        rule = f"If someone buys {antecedent}, they are likely to buy {consequent}." 
 
        print("\n Rule:", rule) 
        explanation = client.models.generate_content(model = model, contents = f"Explain this rule: {rule}") 
        display(Markdown(f"**SLM Explanation:** {explanation.text}"))
else: 
        print("\n No association rules found. Try further lowering support/confidence thresholds or check your data.")


 Rule: If someone buys Bread, they are likely to buy Butter.


**SLM Explanation:** This rule, "If someone buys Bread, they are likely to buy Butter," is a classic example of **market basket analysis** and demonstrates a concept called **association rule mining**. Here's a breakdown of why this rule exists and what it means:

**Why it's true (the reasoning):**

* **Complementary Goods:** Bread and butter are *complementary goods*. This means they are often used *together*.  People commonly eat butter *on* bread.  One enhances the enjoyment of the other.
* **Common Meal/Snack:** They're frequently part of the same meal or snack – toast, sandwiches, etc.
* **Habit & Tradition:**  For many, using butter with bread is a deeply ingrained habit.
* **Convenience:** If someone is already shopping for bread, it's convenient to grab butter at the same time, anticipating needing it.

**What the rule means in a business context:**

* **Retail Strategy:**  Supermarkets and grocery stores use this information to optimize product placement.  You'll often find bread and butter located near each other. This encourages impulse purchases.
* **Promotions & Bundling:**  They might run promotions like "Buy bread, get butter 50% off" or bundle them together as a package deal.
* **Inventory Management:** Knowing this association helps stores predict demand. If bread sales are high, they can anticipate increased butter sales and adjust inventory accordingly.
* **Marketing:**  Targeted advertising – if someone buys bread online, they might see ads for butter.
* **Recommendation Systems:**  If you add bread to your online shopping cart, the website might suggest adding butter.



**In essence, the rule isn't saying *everyone* who buys bread buys butter. It's saying there's a statistically significant *correlation* between the two purchases.**  The strength of the rule is measured by metrics like:

* **Support:** How often both items are purchased together (as a percentage of all transactions).
* **Confidence:**  Of all the people who buy bread, what percentage *also* buy butter? (This is what the rule directly states).
* **Lift:**  How much more likely are people to buy butter when they buy bread, compared to how likely they are to buy butter in general? (A lift of >1 indicates a positive association).



This simple rule is a foundational concept in data mining and is used extensively in retail and other industries to understand customer behavior and improve business strategies.






 Rule: If someone buys Butter, they are likely to buy Bread.


**SLM Explanation:** This rule, "If someone buys Butter, they are likely to buy Bread," is a classic example of **market basket analysis** and demonstrates a concept called **association**. It's based on the observation of **customer purchasing patterns**. Here's a breakdown of why this rule exists and what it means:

* **Common Usage:** Butter and bread are frequently used *together*.  People typically use butter *on* bread – for toast, sandwiches, or simply spreading on a slice.  It's a very common food pairing.
* **Complementary Goods:**  Economically, butter and bread are considered **complementary goods**. This means the demand for one good (butter) increases the demand for the other (bread), and vice versa.  If you're planning to have toast, you'll likely buy both butter *and* bread.
* **Data-Driven Discovery:** This rule isn't just a guess. It's usually discovered by analyzing large datasets of transaction data (like from a grocery store's point-of-sale system).  The analysis shows that a statistically significant number of transactions *include both* butter and bread.
* **Association Rule Mining:**  The process of finding these kinds of relationships is called **association rule mining**. Algorithms look for items that frequently appear together in transactions.  The rule is often expressed with metrics like:
    * **Support:** How often both items appear together in all transactions.
    * **Confidence:**  Of all the transactions that contain butter, what percentage *also* contain bread? (This is the "likely" part of the rule).
    * **Lift:**  How much more likely is bread to be purchased when butter is purchased, compared to the general probability of buying bread? (A lift > 1 indicates a positive association).

**How it's used in practice:**

* **Product Placement:** Grocery stores will often place butter and bread near each other in the store to encourage impulse purchases.
* **Promotions:**  They might run promotions like "Buy butter, get bread at a discount" to capitalize on this association.
* **Recommendations:** Online retailers might suggest bread to customers who have added butter to their cart ("Customers who bought this item also bought...").
* **Inventory Management:**  Knowing this relationship helps stores manage their inventory levels to ensure they have enough of both items in stock.



In short, the rule isn't a law of nature, but a statistically observed tendency based on how people typically use these products. It's a valuable insight for businesses to understand customer behavior and optimize their sales strategies.






 Rule: If someone buys Coke, they are likely to buy Bread.


...

**SLM Explanation:** This rule is a classic example of **market basket analysis** and demonstrates a concept called **association rule mining**. It essentially means that there's a statistically significant correlation between the purchase of Bread and Butter and the purchase of Milk. 

Here's a breakdown of why this rule exists and what it means:

* **Common Consumption Patterns:** Bread and Butter are often eaten *with* Milk.  Think about breakfast, tea time, or a simple snack. They're frequently consumed together as a meal or part of one.
* **Complementary Goods:**  Bread, Butter, and Milk are considered **complementary goods**.  This means the usefulness or enjoyment of one good increases when the others are also present.  Bread and butter are good, but *better* with a glass of milk.
* **Data-Driven Discovery:** This rule isn't based on someone's intuition necessarily. It's discovered by analyzing large datasets of transaction data (like supermarket receipts).  The analysis shows that a significant percentage of transactions containing both Bread *and* Butter *also* contain Milk.
* **Association, Not Causation:**  It's important to understand this is an *association*, not a *causation*. Buying bread and butter doesn't *cause* someone to buy milk. It's simply that people who buy bread and butter are *more likely* to also buy milk.

**How it's used in practice:**

Retailers use rules like this for:

* **Product Placement:** Placing Milk near the Bread and Butter sections in the store.
* **Promotions:** Offering discounts on Milk when customers buy Bread and Butter. ("Buy Bread & Butter, get Milk 50% off!")
* **Recommendations:**  Suggesting Milk to customers who have Bread and Butter in their online shopping cart. ("Customers who bought this also bought...")
* **Inventory Management:** Ensuring sufficient stock of Milk when Bread and Butter sales are high.



In short, the rule "If someone buys Bread, Butter, they are likely to buy Milk" is a valuable insight into consumer behavior that retailers can leverage to increase sales and improve the shopping experience.






 Rule: If someone buys Milk, they are likely to buy Bread, Butter.


**SLM Explanation:** This rule is a classic example of **market basket analysis** and demonstrates a concept called **association rule mining**. It's based on the observation that certain items are frequently purchased *together*. Here's a breakdown of why this rule ("If someone buys Milk, they are likely to buy Bread, Butter") exists and what it means:

**Why it happens (the underlying reasons):**

* **Common Breakfast/Meal Combinations:** Milk, bread, and butter are all staples of a very common breakfast – toast with butter and a glass of milk.  They also fit into other meals like sandwiches.
* **Household Staples:** These are all frequently used items in many households. People often replenish them at the same time.
* **Habitual Buying:**  People get into routines. If they're already shopping for milk, it's easy to remember they also need bread and butter.
* **Complementary Goods:** They *complement* each other.  Bread and butter are often used *with* milk (or other beverages). Milk is often used *on* cereal which is often eaten *with* bread.

**What the rule means (in a retail/business context):**

* **Predictive Power:**  The rule suggests that if a customer has milk in their shopping cart, there's a higher probability they will also purchase bread and butter.
* **Cross-Selling Opportunities:** Retailers can use this information to:
    * **Placement:** Place bread and butter near the milk in the store.
    * **Promotions:** Offer discounts or bundles (e.g., "Buy milk and get a discount on bread and butter").
    * **Recommendations:**  If a customer is buying milk online, suggest bread and butter as "Frequently Bought Together" items.
* **Inventory Management:**  Knowing these items are often bought together helps retailers manage their inventory levels to avoid stockouts.



**In essence, this rule isn't about a *causal* relationship (milk *causes* you to buy bread and butter). It's about a *correlation* – a statistical relationship showing that these items frequently appear together in customer purchases.**



This type of analysis is widely used in retail, e-commerce, and other industries to understand customer behavior and optimize sales strategies.  It's a fundamental part of data mining and machine learning.






 Rule: If someone buys Butter, they are likely to buy Bread, Milk.


**SLM Explanation:** This rule is a classic example of **market basket analysis** or **association rule mining**. It's based on the observation of purchasing patterns – specifically, that there's a strong *correlation* between buying butter and buying bread and milk.  Here's a breakdown of what it means and why it happens:

**What it means:**

* **If...Then Rule:** The rule states that *if* a customer puts butter in their shopping basket, *then* there's a high probability they will *also* purchase bread and milk.
* **Not Causation, but Correlation:**  It's crucial to understand this isn't saying butter *causes* people to buy bread and milk. It simply means these items are frequently bought *together*.
* **Likelihood, Not Guarantee:** It doesn't mean *every* butter buyer will buy bread and milk. It means the *likelihood* is significantly higher than if they hadn't bought butter.

**Why this happens (the underlying reasons):**

* **Common Use Case:** Butter, bread, and milk are all staples in a very common breakfast or snack scenario: toast with butter and a glass of milk.  This is the most obvious and likely driver.
* **Sandwiches:** Butter and bread are essential for making sandwiches, and milk is often consumed with sandwiches.
* **Baking:** All three ingredients are frequently used in baking (cakes, cookies, etc.).
* **Habitual Purchases:** People often buy these items together out of habit.  If they regularly have toast for breakfast, they'll likely buy all three items on a grocery trip.
* **Meal Planning:**  Customers might be planning a meal (like French toast) that requires all three ingredients.



**How this rule is used (in practice):**

* **Retail Store Layout:** Stores often place these items near each other to encourage impulse purchases.  You'll frequently find bread and milk close to the butter section.
* **Promotions & Bundling:**  Retailers might offer promotions like "Buy Butter, Get a Discount on Bread and Milk" or bundle them together.
* **Targeted Advertising:**  If a customer buys butter online, the website might suggest bread and milk as "You might also like..." items.
* **Inventory Management:**  Knowing this association helps stores ensure they have enough stock of all three items, especially during peak shopping times.
* **Recommendation Systems:**  Used in online shopping to suggest related products.



**In short, the rule highlights a common consumer behavior pattern that retailers can leverage to increase sales and improve the shopping experience.**



